# **Python Imports & Settings**

In [1]:
import os
import sys
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if base_path not in sys.path:
    sys.path.append(base_path)

from src.utils.class_lib import HDF5Dataset
from src.utils.class_lib import FrameTargetDataset
from src.utils import lib

import numpy as np
import napari
import PyQt6

import matplotlib.pyplot as plt
import json


In [2]:
with open(f"{base_path}/config/parameters.json", "r") as file:
        parametri = json.load(file)
        
mode  = parametri["setting"]["mode"]
rseed = parametri["setting"]["rseed"]

# Paths
mode_path = base_path + '/data/' + mode

pkl_frame_res_path    = mode_path + '/frames-study.pkl'
dataset_h5_path       = mode_path + '/dataset.h5'
pkl_hospitaldict_path = mode_path + '/hospitals-patients-dict.pkl'

# **Getting Data**

In [3]:
# Create the custom dataset
try:
    del(dataset)
    dataset = HDF5Dataset(dataset_h5_path)
except:
    dataset = HDF5Dataset(dataset_h5_path)

Calculating frames and index map:   0%|          | 0/230 [00:00<?, ?video/s]

Calculating frames and index map: 100%|██████████| 230/230 [00:00<00:00, 3886.63video/s]


230 videos (2154 frames) loaded.


In [4]:
train_subset, test_subset, split_info = lib.splitting_strategy(dataset, rseed, pkl_frame_res_path, 0.7)

# Print diagnostic information
for medical_center in split_info['medical_center_patients'].keys():
    print(f"Medical Center: {medical_center}")
    print(f"  Frames in center: {split_info['frame_counts_by_center'][medical_center]}")
    print(f"  Train patients:")
    for patient in split_info['train_patients_by_center'][medical_center]:
        frame_count = split_info['frame_counts_by_center_patient'][medical_center][patient]
        print(f"   {patient}: {frame_count} frames")
    print(f"  Test patients:")
    for patient in split_info['test_patients_by_center'][medical_center]:
        frame_count = split_info['frame_counts_by_center_patient'][medical_center][patient]
        print(f"   {patient}: {frame_count} frames")

total_train_frames = split_info['total_train_frames']
total_test_frames = split_info['total_test_frames']
print(f"\nTotal Train Frames: {total_train_frames} ({round(total_train_frames*100/len(dataset))}%)")
print(f"Total Test Frames: {total_test_frames} ({round(total_test_frames*100/len(dataset))}%)")

Medical Center: Brescia
  Frames in center: 1410
  Train patients:
   Paziente 8: 129 frames
   Paziente 1: 9 frames
   Paziente 3: 111 frames
   Paziente 2: 78 frames
   Paziente 4: 116 frames
   Paziente 7: 136 frames
   Paziente 13: 81 frames
   Paziente 6: 200 frames
   Paziente 5: 60 frames
  Test patients:
   Paziente 10: 153 frames
   Paziente 11: 98 frames
   Paziente 12: 84 frames
   Paziente 9: 155 frames
Medical Center: Gemelli - Roma
  Frames in center: 158
  Train patients:
   Paziente 1: 116 frames
   Paziente 2: 39 frames
  Test patients:
   Paziente 3: 3 frames
Medical Center: Lucca
  Frames in center: 118
  Train patients:
   Paziente 1: 51 frames
  Test patients:
   Paziente 2: 67 frames
Medical Center: No Covid Data
  Frames in center: 400
  Train patients:
   Paziente 8: 42 frames
   Paziente 12: 33 frames
   Paziente 2: 58 frames
   Paziente 11: 51 frames
   Paziente 4: 16 frames
   Paziente 13: 13 frames
   Paziente 6: 11 frames
   Paziente 10: 14 frames
   Pazien

In [5]:
train_dataset = FrameTargetDataset(train_subset)
test_dataset  = FrameTargetDataset(test_subset)

# **Napari Visualization**

In [ ]:
# Carica il tuo dataset e organizza i frame come un array tridimensionale
frames = []  # Lista di frame
frames_mask = []
for frame_tensor, mask_tensor in test_dataset:
    frames.append(frame_tensor.numpy())  # Converte il tensore in un array numpy

    mask_rgb = np.zeros((224, 224, 3), dtype=np.uint8)  # Crea un'immagine RGB vuota
    mask_rgb[mask_tensor.numpy()[:, :, 0] >= 0] = [255, 255, 255]  # Rosso (R, G, B)
    
    frames_mask.append(mask_rgb)  # Converte il tensore in un array numpy

# Crea un array numpy tridimensionale (tempo, altezza, larghezza)
video = np.stack(frames)
video_mask = np.stack(frames_mask)

video = video[:5,:,:,:]
video_mask = video_mask[:5,:,:,:]



In [ ]:
viewer = napari.Viewer()
# Aggiungi il video a Napari
#viewer.add_image(video, name='Video') 
viewer.add_labels(video_mask, name='Video') 

napari.run()